# ETL Credits:

## Libreries

In [1]:
import pandas as pd
import ast
import missingno as msno
import json


## Opening CSV Credits data:

In [2]:
df_credits_csv = pd.read_csv('D:\Data Scientist\CURSADO HENRY DTS09\PI\PI1_PT02\PI_ML_OPS_PT02\Datasets\Credits_datasets\Csvs Credits\credits_1.csv')

In [3]:
df_credits_csv.head(1)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


### Vamos a tener que explorarlo mas al apestoso csv credits 🤧

In [4]:
df_credits_csv.shape

(45476, 3)

In [5]:
df_credits_csv = df_credits_csv.drop_duplicates()

In [6]:
df_credits_csv.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45439 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45439 non-null  object
 1   crew    45439 non-null  object
 2   id      45439 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


elimino la ultima fila que esta vacia

In [7]:
df_credits_csv = df_credits_csv.drop(df_credits_csv.index[-1])

### Separo la columna id y la guardo aparte para asegurarla, debido a que parece estar ok

In [8]:
df_movie_credits_id = pd.DataFrame()
df_movie_credits_id['id']= df_credits_csv['id']
df_movie_credits_id = df_movie_credits_id.drop_duplicates()
print(type(df_movie_credits_id))

<class 'pandas.core.frame.DataFrame'>


### Continuo Explorando las dos primeras columnas, cast y crew

#### Cast:

In [9]:
df_credits_cast = df_credits_csv[['cast','id']]

print(df_credits_cast)

                                                    cast      id
0      [{'cast_id': 14, 'character': 'Woody (voice)',...     862
1      [{'cast_id': 1, 'character': 'Alan Parrish', '...    8844
2      [{'cast_id': 2, 'character': 'Max Goldman', 'c...   15602
3      [{'cast_id': 1, 'character': "Savannah 'Vannah...   31357
4      [{'cast_id': 1, 'character': 'George Banks', '...   11862
...                                                  ...     ...
45470  [{'cast_id': 1, 'character': 'Sir Robert Hode'...   30840
45471  [{'cast_id': 0, 'character': '', 'credit_id': ...  439050
45472  [{'cast_id': 1002, 'character': 'Sister Angela...  111109
45473  [{'cast_id': 6, 'character': 'Emily Shaw', 'cr...   67758
45474  [{'cast_id': 2, 'character': '', 'credit_id': ...  227506

[45438 rows x 2 columns]


In [10]:
df_credits_cast = df_credits_cast.drop_duplicates()

In [11]:
print(df_credits_cast.iloc[1:2])

                                                cast    id
1  [{'cast_id': 1, 'character': 'Alan Parrish', '...  8844


In [12]:
iteracion = 0
df_cast_cleaned = pd.DataFrame()

# Iterar a través de la columna 'cast' en df_credits_cast
for row in df_credits_cast['cast']:
    try:
        # Utilizar json.loads para convertir la representación de cadena en una lista de diccionarios
        list_of_dicts = json.loads(row.replace("'", "\""))

        # Verificar si el 'id' de la película existe en la columna 'id' de df_movie_credits_id
        if df_movie_credits_id['id'].isin([d['id'] for d in list_of_dicts]).any():
            # Crear el DataFrame con los datos del cast
            cast_df = pd.DataFrame(list_of_dicts)

            # Obtener el 'id' de la película de df_movie_credits_id
            movie_id = df_movie_credits_id['id'].iloc[iteracion]

            # Agregar el 'movie_id' al DataFrame cast_df
            cast_df['movie_id'] = movie_id

            # Unificar los DataFrames utilizando concat
            df_cast_cleaned = pd.concat([df_cast_cleaned, cast_df], ignore_index=True)

            # Incrementar el contador de iteración
            iteracion += 1
        else:
            # Si el 'id' de la película no existe en df_movie_credits_id, omitimos esta película
            pass
    except json.JSONDecodeError:
        # Omitir las filas que no se pueden decodificar como JSON
        pass


In [13]:
cast_df

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,1,Sir Robert Hode,52fe44439251416c9100a887,2,29459,Patrick Bergin,0,/mxFum2Fr5YWD6kDQYb5u7EEvuva.jpg,111815
1,2,Maid Marian,52fe44439251416c9100a88b,1,139,Uma Thurman,1,/6SuOc2R7kXjq3Em24KTNDW9qblJ.jpg,111815
2,3,Little John,52fe44439251416c9100a88f,2,18616,David Morrissey,2,/y8L9zlrIzsAwu2Hufol6KqoDS4I.jpg,111815
3,4,Sir Miles Folcanet,52fe44439251416c9100a893,2,920,Jürgen Prochnow,3,/aar3n9NEQdwWVvPOne3kLLXm75N.jpg,111815
4,14,Baron Roger Daguerre,58207d0d92514168ad013b03,0,1924,Jeroen Krabbé,4,/A5Mks6zi6Xzt9A8fxBWc3wdlvhx.jpg,111815


In [14]:
df_cast_cleaned.shape

(53558, 9)

In [15]:
df_cast_cleaned

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
0,2,Max Goldman,52fe466a9251416c75077a8d,2,6837,Walter Matthau,0,/xJVkvprOnzP5Zdh5y63y8HHniDZ.jpg,862
1,3,John Gustafson,52fe466a9251416c75077a91,2,3151,Jack Lemmon,1,/chZmNRYMtqkiDlatprGDH4BzGqG.jpg,862
2,4,Ariel Gustafson,52fe466a9251416c75077a95,1,13567,Ann-Margret,2,/jx5lTaJ5VXZHYB52gaOTAZ9STZk.jpg,862
3,5,Maria Sophia Coletta Ragetti,52fe466a9251416c75077a99,1,16757,Sophia Loren,3,/emKLhbji1c7BjcA2DdbWf0EP9zH.jpg,862
4,6,Melanie Gustafson,52fe466a9251416c75077a9d,1,589,Daryl Hannah,4,/4LLmp6AQdlj6ueGCRbVRSGvvFSt.jpg,862
...,...,...,...,...,...,...,...,...,...
53553,1,Sir Robert Hode,52fe44439251416c9100a887,2,29459,Patrick Bergin,0,/mxFum2Fr5YWD6kDQYb5u7EEvuva.jpg,111815
53554,2,Maid Marian,52fe44439251416c9100a88b,1,139,Uma Thurman,1,/6SuOc2R7kXjq3Em24KTNDW9qblJ.jpg,111815
53555,3,Little John,52fe44439251416c9100a88f,2,18616,David Morrissey,2,/y8L9zlrIzsAwu2Hufol6KqoDS4I.jpg,111815
53556,4,Sir Miles Folcanet,52fe44439251416c9100a893,2,920,Jürgen Prochnow,3,/aar3n9NEQdwWVvPOne3kLLXm75N.jpg,111815


In [16]:
df_cast_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53558 entries, 0 to 53557
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cast_id       53558 non-null  int64 
 1   character     53558 non-null  object
 2   credit_id     53558 non-null  object
 3   gender        53558 non-null  int64 
 4   id            53558 non-null  int64 
 5   name          53558 non-null  object
 6   order         53558 non-null  int64 
 7   profile_path  53558 non-null  object
 8   movie_id      53558 non-null  int64 
dtypes: int64(5), object(4)
memory usage: 3.7+ MB


##### Confirmacion de Diector id:

In [17]:
# Filtrar el DataFrame por el valor 138 en la columna "id"
df_filtrado = df_cast_cleaned[df_cast_cleaned["id"] == 138].copy()


In [18]:
df_filtrado.tail(50)

,cast_id,character,credit_id,gender,id,name,order,profile_path,movie_id
405,5,Destiny,52fe4680c3a368484e094179,2,138,Quentin Tarantino,3,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,35196
1233,13,Himself,5452d51bc3a36809270004de,2,138,Quentin Tarantino,17,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,27793
1330,10,Bartender,5990dcc3c3a36874c4037865,2,138,Quentin Tarantino,6,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,9302
2818,6,Himself,52fe45f89251416c91044ae5,2,138,Quentin Tarantino,4,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,109
5273,25,Deacon,52fe451ac3a36847f80bc92f,2,138,Quentin Tarantino,16,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,319
9831,7,Sid,52fe47d39251416c750a6a1d,2,138,Quentin Tarantino,5,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,454
14458,3,Quentin Tarantino,52fe47d79251416c750a735b,2,138,Quentin Tarantino,2,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,10351
16048,30,Warren,52fe4329c3a36847f803f111,2,138,Quentin Tarantino,9,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,36568
16333,14,The Rapist,52fe4329c3a36847f803f18f,2,138,Quentin Tarantino,13,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,47881
17558,1001,Himself,536ffa830e0a2647d4012eab,2,138,Quentin Tarantino,1,/9ci4NBvHXJktxjALdJsrtasqgtV.jpg,11485


#### Crew:

In [19]:
df_credits_crew = df_credits_csv[['crew','id']]

print(df_credits_crew)

                                                    crew      id
0      [{'credit_id': '52fe4284c3a36847f8024f49', 'de...     862
1      [{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...    8844
2      [{'credit_id': '52fe466a9251416c75077a89', 'de...   15602
3      [{'credit_id': '52fe44779251416c91011acb', 'de...   31357
4      [{'credit_id': '52fe44959251416c75039ed7', 'de...   11862
...                                                  ...     ...
45470  [{'credit_id': '52fe44439251416c9100a899', 'de...   30840
45471  [{'credit_id': '5894a97d925141426c00818c', 'de...  439050
45472  [{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...  111109
45473  [{'credit_id': '52fe4776c3a368484e0c8387', 'de...   67758
45474  [{'credit_id': '533bccebc3a36844cf0011a7', 'de...  227506

[45438 rows x 2 columns]


In [20]:
df_credits_cast = df_credits_cast.drop_duplicates()

In [21]:
iteracion = 0
df_crew_cleaned = pd.DataFrame()

# Iterar a través de la columna 'cast' en df_credits_cast
for row in df_credits_crew['crew']:
    try:
        # Utilizar json.loads para convertir la representación de cadena en una lista de diccionarios
        list_of_dicts = json.loads(row.replace("'", "\""))

        # Verificar si el 'id' de la película existe en la columna 'id' de df_movie_credits_id
        if df_movie_credits_id['id'].isin([d['id'] for d in list_of_dicts]).any():
            # Crear el DataFrame con los datos del cast
            crew_df = pd.DataFrame(list_of_dicts)

            # Obtener el 'id' de la película de df_movie_credits_id
            movie_id = df_movie_credits_id['id'].iloc[iteracion]

            # Agregar el 'movie_id' al DataFrame cast_df
            crew_df['movie_id'] = movie_id

            # Unificar los DataFrames utilizando concat
            df_crew_cleaned = pd.concat([df_crew_cleaned, crew_df], ignore_index=True)

            # Incrementar el contador de iteración
            iteracion += 1
        else:
            # Si el 'id' de la película no existe en df_movie_credits_id, omitimos esta película
            pass
    except json.JSONDecodeError:
        # Omitir las filas que no se pueden decodificar como JSON
        pass


In [22]:
crew_df

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,5330178a9251413a720055e3,Writing,2,20875,Novel,Leo Tolstoy,/9aao2Es6ciI1LmGPH7ZptPfS7fK.jpg,266022
1,52fe475bc3a368484e0c2a87,Writing,2,24279,Screenplay,Paolo Taviani,/su9WmmAKq43ICIFxRlbAQIE3qwQ.jpg,266022
2,52fe475bc3a368484e0c2a7b,Directing,2,24279,Director,Paolo Taviani,/su9WmmAKq43ICIFxRlbAQIE3qwQ.jpg,266022
3,52fe475bc3a368484e0c2a81,Directing,0,24281,Director,Vittorio Taviani,/m6LbjdSX1gBkckAHxtvM4TSESsy.jpg,266022


In [23]:
df_crew_cleaned.shape

(2789, 8)

In [24]:
df_crew_cleaned

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,52fe44669251416c9100f5a3,Directing,2,33315,Director,Richard Loncraine,/j700yY5SyI1EZegFpL4h77Otu5W.jpg,862
1,546110c3c3a3686f31000401,Writing,2,6210,Theatre Play,William Shakespeare,/2z4njqosEQM4g26ttb9grG9rJUg.jpg,862
2,546110d00e0a2672aa0003d7,Writing,2,1327,Writer,Ian McKellen,/coWjgMEYJjk2OrNddlXCBm8EIr3.jpg,862
3,546110dcc3a3686f350003e8,Writing,2,33315,Writer,Richard Loncraine,/j700yY5SyI1EZegFpL4h77Otu5W.jpg,862
4,52fe43f2c3a368484e007465,Directing,2,5281,Director,Spike Lee,/zTsaE7GrLnOV62NmSrr6hll1G5Y.jpg,8844
...,...,...,...,...,...,...,...,...
2784,52fe4a71c3a36847f81ce35b,Directing,0,11523,Director,Georges Méliès,/cjQ3IvPbkNArGjhj4dkGqEwOgdW.jpg,639
2785,5330178a9251413a720055e3,Writing,2,20875,Novel,Leo Tolstoy,/9aao2Es6ciI1LmGPH7ZptPfS7fK.jpg,266022
2786,52fe475bc3a368484e0c2a87,Writing,2,24279,Screenplay,Paolo Taviani,/su9WmmAKq43ICIFxRlbAQIE3qwQ.jpg,266022
2787,52fe475bc3a368484e0c2a7b,Directing,2,24279,Director,Paolo Taviani,/su9WmmAKq43ICIFxRlbAQIE3qwQ.jpg,266022


In [25]:
df_crew_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2789 entries, 0 to 2788
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   credit_id     2789 non-null   object
 1   department    2789 non-null   object
 2   gender        2789 non-null   int64 
 3   id            2789 non-null   int64 
 4   job           2789 non-null   object
 5   name          2789 non-null   object
 6   profile_path  2789 non-null   object
 7   movie_id      2789 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 174.4+ KB


#### No todas las movies poseen un mismo director...:

In [26]:
# Crear un diccionario para almacenar la cantidad de directores por movie_id
directores_por_movie_id = {}

# Recorrer el dataframe y contar los directores para cada movie_id
for index, row in df_crew_cleaned.iterrows():
    if row['job'] == 'Director':
        movie_id = row['movie_id']
        directores_por_movie_id[movie_id] = directores_por_movie_id.get(movie_id, 0) + 1

# Verificar si hay movie_id con más de dos directores
for movie_id, num_directores in directores_por_movie_id.items():
    if num_directores > 2:
        print(f"El movie_id {movie_id} tiene {num_directores} directores.")


El movie_id 13701 tiene 4 directores.
El movie_id 10731 tiene 3 directores.
El movie_id 49803 tiene 7 directores.
El movie_id 507 tiene 8 directores.
El movie_id 812 tiene 3 directores.
El movie_id 117730 tiene 3 directores.
El movie_id 44535 tiene 9 directores.
El movie_id 804 tiene 3 directores.
El movie_id 54405 tiene 3 directores.
El movie_id 756 tiene 3 directores.


#### Listado unico de job == Director ; directors_df 

In [27]:
# Filtrar el DataFrame original para obtener solo las filas con la categoría "Director"
directors_df = df_crew_cleaned[df_crew_cleaned['job'] == 'Director']

# Crear un nuevo DataFrame que contenga solo la columna 'id' (los IDs de los directores)
directors_id_list = directors_df[['id']]

In [28]:
directors_df

,credit_id,department,gender,id,job,name,profile_path,movie_id
0,52fe44669251416c9100f5a3,Directing,2,33315,Director,Richard Loncraine,/j700yY5SyI1EZegFpL4h77Otu5W.jpg,862
4,52fe43f2c3a368484e007465,Directing,2,5281,Director,Spike Lee,/zTsaE7GrLnOV62NmSrr6hll1G5Y.jpg,8844
8,52fe4846c3a36847f815f50f,Directing,1,3092,Director,Diane Keaton,/fzgUMnbOkxC6E3EFcYHWHFaiKyp.jpg,15602
9,52fe471ac3a368484e0b5209,Directing,2,1723,Director,Walter Hill,/gCU3dgnNaZuvmArtBJgjCUMmimd.jpg,31357
11,52fe461d9251416c7506d919,Directing,2,32593,Director,Steven Brill,/9CpoLVxDKV2XRF3rigTNSVimJK1.jpg,11862
...,...,...,...,...,...,...,...,...
2782,52fe4a74c3a368484e1542e9,Directing,0,11523,Director,Georges Méliès,/cjQ3IvPbkNArGjhj4dkGqEwOgdW.jpg,655
2783,52fe478dc3a36847f813bd6b,Directing,0,11523,Director,Georges Méliès,/cjQ3IvPbkNArGjhj4dkGqEwOgdW.jpg,10341
2784,52fe4a71c3a36847f81ce35b,Directing,0,11523,Director,Georges Méliès,/cjQ3IvPbkNArGjhj4dkGqEwOgdW.jpg,639
2787,52fe475bc3a368484e0c2a7b,Directing,2,24279,Director,Paolo Taviani,/su9WmmAKq43ICIFxRlbAQIE3qwQ.jpg,266022


______________________________________________________________________